In [2]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl import drawing 
import openpyxl

In [3]:
df = pd.read_excel("../input/inputSM_2025-2026.xlsx", "input")
wb = load_workbook(filename="../templates/schedule_template.xlsx")

In [4]:
df["sem"] = df.semaine.apply(lambda x: "a" if "A" in x else "b")

In [5]:
template = wb["template"]

In [6]:
# get all tags in sheet
tags = {}
for row in template.iter_rows():
    for cell in row:
        if cell.value:
            if "{" in cell.value:
                tags[cell.value] = {
                    "value" : "", # free all tags content
                    "col" : cell.column,
                    "row": cell.row
                }

In [7]:
print(tags)

{'{classe}': {'value': '', 'col': 4, 'row': 3}, '{j1p1ac}': {'value': '', 'col': 4, 'row': 8}, '{j1p1bc}': {'value': '', 'col': 5, 'row': 8}, '{j2p1ac}': {'value': '', 'col': 6, 'row': 8}, '{j2p1bc}': {'value': '', 'col': 7, 'row': 8}, '{j3p1ac}': {'value': '', 'col': 8, 'row': 8}, '{j3p1bc}': {'value': '', 'col': 9, 'row': 8}, '{j4p1ac}': {'value': '', 'col': 10, 'row': 8}, '{j4p1bc}': {'value': '', 'col': 11, 'row': 8}, '{j5p1ac}': {'value': '', 'col': 12, 'row': 8}, '{j5p1bc}': {'value': '', 'col': 13, 'row': 8}, '{j1p1ap}': {'value': '', 'col': 4, 'row': 9}, '{j1p1bp}': {'value': '', 'col': 5, 'row': 9}, '{j2p1ap}': {'value': '', 'col': 6, 'row': 9}, '{j2p1bp}': {'value': '', 'col': 7, 'row': 9}, '{j3p1ap}': {'value': '', 'col': 8, 'row': 9}, '{j3p1bp}': {'value': '', 'col': 9, 'row': 9}, '{j4p1ap}': {'value': '', 'col': 10, 'row': 9}, '{j4p1bp}': {'value': '', 'col': 11, 'row': 9}, '{j5p1ap}': {'value': '', 'col': 12, 'row': 9}, '{j5p1bp}': {'value': '', 'col': 13, 'row': 9}, '{j1

In [8]:
classes = df.classe.unique()

In [9]:
for cl in classes :
    # get the data
    classe = df[df.classe == cl].copy()

    # get adequate template
    tpl = "template"
    if "P0" in classe.plage.values:
        tpl = "template P0"
    if "P25" in classe.plage.values:
        tpl = "template P25"
    template = wb[tpl]
    
    # copy template
    sheet = wb.copy_worksheet(template)
    sheet.title = cl
    
    # get all tags from template
    tags = {}
    for row in template.iter_rows():
        for cell in row:
            if cell.value:
                if "{" in cell.value:
                    tags[cell.value] = {
                        "value" : "", # free all tags content
                        "col" : cell.column,
                        "row": cell.row
                    }


    # add logo
    img = drawing.image.Image('../templates/Logo Ecole .jpg')
    scale = 0.1
    img.width = img.width * scale
    img.height = img.height * scale
    sheet.add_image(img, 'A1')

    # set title
    tags["{title}"]["value"] = cl

    # Populate the tags
    for i, r in classe.iterrows():
        
        plage = ""
        if r["plage"] in ["P00", "P01"]:
            plage = f"({r.start} - {r.end})"
        if r["plage"] == "P00":
            r["plage"] = "P0"
        elif r["plage"] == "P01":
            r["plage"] = "P1"
            
        t = f"{{j{r['#day']}{r.plage.lower()}{r['sem']}"

        tags[t+"c}"]["value"] = r['cours'].upper()
        tags[t+"p}"]["value"] = r["pro"]
        tags[t+"l}"]["value"] = r["local"]
        tags[t+"o}"]["value"] = plage
        

    # populate the tags in the new sheet
    for t in tags:
        sheet.cell(row=tags[t]['row'], column=tags[t]['col']).value = tags[t]['value']

    # Merge cells if needed
    for i, r in classe.iterrows():

        plage = ""
        if r["plage"] in ["P00", "P01"]:
            plage = f"({r.start} - {r.end})"
        if r["plage"] == "P00":
            r["plage"] = "P0"
        elif r["plage"] == "P01":
            r["plage"] = "P1"
        
        t = f"{{j{r['#day']}{r.plage.lower()}{r['sem']}"
        
        # if AB course, get the corresponding b tag for merging
        if r.semaine == 'AB':
            tb = f"{{j{r['#day']}{r.plage.lower()}b"
            for j in ['c', 'p', 'o', 'l']:
                sheet.merge_cells(
                    start_row=tags[t+j+"}"]['row'],
                    start_column=tags[t+j+"}"]['col'],
                    end_row=tags[tb+j+"}"]['row'],
                    end_column=tags[tb+j+"}"]['col']
                )  

In [10]:
for s in ["template","template P0", "template P25", "template P025"]:
    ws = wb[s]
    wb.remove(ws)

In [11]:
# Save result
wb.save("../output/horaire_classes.xlsx")

In [33]:
wb = load_workbook(filename="../templates/schedule_template.xlsx")

In [34]:
pros = df.pro.unique()

In [35]:
pro = df[df.pro == "maxime.marlier"].copy()

In [36]:
    # # Populate the tags
    # for i, r in pro.iterrows():
        
    #     plage = ""
    #     if r["plage"] in ["P00", "P01"]:
    #         plage = f"({r.start} - {r.end})"
    #     if r["plage"] == "P00":
    #         r["plage"] = "P0"
    #     elif r["plage"] == "P01":
    #         r["plage"] = "P1"
            
    #     t = f"{{j{r['#day']}{r.plage.lower()}{r['sem']}"

    #     tags[t+"c}"]["value"] = r['cours'].upper()
    #     tags[t+"p}"]["value"] = r["pro"]
    #     tags[t+"l}"]["value"] = r["local"]
    #     tags[t+"o}"]["value"] = r["code horaire"]

" ; ".join(pro[(pro['#day'] == 4 ) & (pro.plage == "P4") & (pro.semaine == "AB")]["code horaire"])

'5A-SI ; 5B-SI ; 6B-SI'

In [37]:
for prof in pros :
    # get the data
    pro = df[df.pro == prof].copy()

    # get adequate template
    tpl = "template"
    if "P0" in pro.plage.values:
        tpl = "template P0"
    if "P25" in pro.plage.values:
        tpl = "template P025"
    template = wb[tpl]
    
    # copy template
    sheet = wb.copy_worksheet(template)
    sheet.title = prof
    
    # get all tags from template
    tags = {}
    for row in template.iter_rows():
        for cell in row:
            if cell.value:
                if "{" in cell.value:
                    tags[cell.value] = {
                        "value" : "", # free all tags content
                        "col" : cell.column,
                        "row": cell.row
                    }


    # add logo
    img = drawing.image.Image('../templates/Logo Ecole .jpg')
    scale = 0.1
    img.width = img.width * scale
    img.height = img.height * scale
    sheet.add_image(img, 'A1')

    # set title
    tags["{classe}"]["value"] = prof

    # Populate the tags
    for i, r in pro.iterrows():
        
        plage = ""
        if r["plage"] in ["P00", "P01"]:
            plage = f"({r.start} - {r.end})"
        if r["plage"] == "P00":
            r["plage"] = "P0"
        elif r["plage"] == "P01":
            r["plage"] = "P1"
            
        t = f"{{j{r['#day']}{r.plage.lower()}{r['sem']}"
        

        tags[t+"c}"]["value"] = r['cours'].upper()
        tags[t+"p}"]["value"] = r["pro"]
        tags[t+"l}"]["value"] = r["local"]
        # tags[t+"o}"]["value"] = r["code horaire"]
        tags[t+"o}"]["value"] = " ; ".join(pro[(pro['#day'] == r['#day'] ) 
                                           & (pro.plage == r.plage) 
                                           & (pro.semaine == r.semaine)]["code horaire"])
        

    # populate the tags in the new sheet
    for t in tags:
        sheet.cell(row=tags[t]['row'], column=tags[t]['col']).value = tags[t]['value']

    # Merge cells if needed
    for i, r in pro.iterrows():

        plage = ""
        if r["plage"] in ["P00", "P01"]:
            plage = f"({r.start} - {r.end})"
        if r["plage"] == "P00":
            r["plage"] = "P0"
        elif r["plage"] == "P01":
            r["plage"] = "P1"
        
        t = f"{{j{r['#day']}{r.plage.lower()}{r['sem']}"
        
        # if AB course, get the corresponding b tag for merging
        if r.semaine == 'AB':
            tb = f"{{j{r['#day']}{r.plage.lower()}b"
            for j in ['c', 'p', 'o', 'l']:
                sheet.merge_cells(
                    start_row=tags[t+j+"}"]['row'],
                    start_column=tags[t+j+"}"]['col'],
                    end_row=tags[tb+j+"}"]['row'],
                    end_column=tags[tb+j+"}"]['col']
                )  

In [38]:
for s in ["template","template P0", "template P25", "template P025"]:
    ws = wb[s]
    wb.remove(ws)

In [39]:
# Save result
wb.save("../output/horaire_pros.xlsx")